In [1]:
# !pip install tqdm
# !pip install torch
# !pip install torchvision torchaudio
# !pip install fastai
# !pip install tensorboardX
# !pip install scikit-learn
# !pip install "git+https://github.com/ncullen93/torchsample.git#egg=torchsample"
# !pip install nibabel
# !pip install nbdev

     |████████████████████████████████| 804.1 MB 3.2 kB/s eta 0:00:011  |                                | 604 kB 2.6 MB/s eta 0:05:07     |███▍                            | 85.9 MB 9.8 MB/s eta 0:01:14     |███▍                            | 86.1 MB 9.8 MB/s eta 0:01:14     |████                            | 101.2 MB 11.7 MB/s eta 0:01:01     |████                            | 103.0 MB 11.7 MB/s eta 0:01:01     |████▋                           | 114.8 MB 10.6 MB/s eta 0:01:06     |█████▎                          | 132.4 MB 13.7 MB/s eta 0:00:50     |███████▋                        | 191.5 MB 6.2 MB/s eta 0:01:39     |████████▌                       | 214.9 MB 10.7 MB/s eta 0:00:55     |█████████▉                      | 247.6 MB 12.7 MB/s eta 0:00:44��████                      | 248.9 MB 12.7 MB/s eta 0:00:44MB/s eta 0:00:52     |█████████████▏                  | 330.8 MB 813 kB/s eta 0:09:42     |█████████████▌                  | 338.6 MB 15.8 MB/s eta 0:00:30     |█████████████▋      

  Cloning https://github.com/ncullen93/torchsample.git to /tmp/pip-install-o5moal9v/torchsample_bd5b938fa6454f44b7f53df9e770bf88
  Running command git clone -q https://github.com/ncullen93/torchsample.git /tmp/pip-install-o5moal9v/torchsample_bd5b938fa6454f44b7f53df9e770bf88


In [2]:
# !pip install torch==1.6.0 torchvision==0.7.0

     |████████████████████████████████| 748.8 MB 11 kB/s s eta 0:00:01   |▌                               | 10.6 MB 5.1 MB/s eta 0:02:24     |▉                               | 20.1 MB 14.6 MB/s eta 0:00:51     |█▍                              | 31.4 MB 13.3 MB/s eta 0:00:54     |██▉                             | 65.5 MB 9.2 MB/s eta 0:01:15     |███▎                            | 77.0 MB 13.2 MB/s eta 0:00:52     |████████                        | 186.4 MB 808 kB/s eta 0:11:36     |████████▋                       | 202.8 MB 735 kB/s eta 0:12:23     |████████▊                       | 203.5 MB 735 kB/s eta 0:12:22     |█████████▎                      | 216.9 MB 11.3 MB/s eta 0:00:47     |██████████                      | 233.6 MB 738 kB/s eta 0:11:38     |██████████                      | 234.2 MB 738 kB/s eta 0:11:37     |███████████▎                    | 262.7 MB 13.0 MB/s eta 0:00:38     |███████████▎                    | 264.8 MB 13.0 MB/s eta 0:00:38     |████████████▎               

In [1]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm
from nbdev import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from dataloader import MRDataset
import model

from sklearn import metrics
from fastai.vision.all import *

In [2]:
# parser.add_argument('-t', '--task', type=str, required=True,
#                     choices=['abnormal', 'acl', 'meniscus'])
# parser.add_argument('-p', '--plane', type=str, required=True,
#                     choices=['sagittal', 'coronal', 'axial'])
# parser.add_argument('--prefix_name', type=str, required=True)
# parser.add_argument('--augment', type=int, choices=[0, 1], default=1)
# parser.add_argument('--lr_scheduler', type=str,
#                     default='plateau', choices=['plateau', 'step'])
# parser.add_argument('--gamma', type=float, default=0.5)
# parser.add_argument('--epochs', type=int, default=50)
# parser.add_argument('--lr', type=float, default=1e-5)
# parser.add_argument('--flush_history', type=int, choices=[0, 1], default=0)
# parser.add_argument('--save_model', type=int, choices=[0, 1], default=1)
# parser.add_argument('--patience', type=int, default=5)
# parser.add_argument('--log_every', type=int, default=100)
# args = parser.parse_args()

class Args:
    def __init__(self):
        self.task = "abnormal"
        self.plane = "sagittal"
        self.prefix_name = "Test"
        self.augment = 1
        self.lr_scheduler = "plateau"
        self.gamma = 0.5
        self.epochs = 1
        self.lr = 1e-5
        self.flush_history = 0
        self.save_model =1
        self.patience = 5
        self.log_every = 100
        
args = Args()

In [3]:
args.task

'abnormal'

In [4]:
log_root_folder = "./logs/{0}/{1}/".format(args.task, args.plane)
if args.flush_history == 1:
    objects = os.listdir(log_root_folder)
    for f in objects:
        if os.path.isdir(log_root_folder + f):
            shutil.rmtree(log_root_folder + f)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)

augmentor = Compose([
    transforms.Lambda(lambda x: torch.Tensor(x)),
    RandomRotate(25),
    RandomTranslate([0.11, 0.11]),
    RandomFlip(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1, 1).permute(1, 0, 2, 3)),
])

train_dataset = MRDataset('./data/', args.task, args.plane, transform=augmentor, train=True)
validation_dataset = MRDataset('./data/', args.task, args.plane, train=False)

# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=1, shuffle=True, num_workers=11, drop_last=False)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=7, drop_last=False)

# validation_loader = torch.utils.data.DataLoader(
#         validation_dataset, batch_size=1, shuffle=-True, num_workers=11, drop_last=False)

validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, num_workers=7, drop_last=False)


mrnet = model.MRNet()
bceloss = nn.BCEWithLogitsLoss()

./data/train-abnormal.csv


In [5]:
dls = DataLoaders(train_loader, validation_loader)

In [12]:
batch = dls.train.one_batch()
print(len(batch))
print(*[batch[i].shape for i in range(2)])

2
torch.Size([1, 32, 3, 256, 256]) torch.Size([1, 2])


In [19]:
torch.Tensor

torch.Tensor

In [20]:
metric = RocAuc()
metric(torch.Tensor([[0,1],[1,0]]), torch.Tensor([[0,1],[1,0]]))

1.0

In [14]:
batch[1]

tensor([[0., 1.]])

In [14]:
doc(Learner)

In [8]:
RocAuc

<function fastai.metrics.RocAuc(axis=-1, average='macro', sample_weight=None, max_fpr=None, multi_class='ovr')>

In [6]:
learn = Learner(dls, mrnet, loss_func=bceloss, metrics=[RocAuc()])

In [7]:
learn.fine_tune(30, freeze_epochs=1)

epoch,train_loss,valid_loss,roc_auc_score,time
0,3.592594,0.540627,0.573053,30:38


epoch,train_loss,valid_loss,roc_auc_score,time
0,0.463580,0.531733,0.589474,30:32
1,0.526282,0.526122,0.548632,25:41
2,0.541170,0.999072,0.413053,28:42
3,0.848215,0.534778,0.638737,29:00
4,0.665666,0.558005,0.399579,42:30
5,0.505960,0.526987,0.625263,32:28
6,0.636220,0.534038,0.691789,34:23
7,0.552062,0.509216,0.759579,25:11
8,0.431055,0.530921,0.658105,23:34
9,0.599188,0.495058,0.807579,24:17


KeyboardInterrupt: 

In [ ]:
learn.fit_one_epoch()